In [1]:
# %load C60_jupyter_imports.py
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import xesmf as xe
from scipy.stats import linregress
import os
import requests
import dask
# Lets import our functions. There are no tests here. Stored separately so as to not make a mess.
# Make sure in right dir first
from eofs.xarray import Eof
os.chdir('/g/data/xv83/np1383/src_CAFE60_eqpac_analysis_code/')
from C60_obs_functions import cut_regrid_reynolds_sst,cut_process_sst_obs_trends,proc_landschutzer,process_co2_land_trends

In [3]:
from Kruskals import Kruskals

In [4]:
import PyCO2SYS

ModuleNotFoundError: No module named 'PyCO2SYS'

In [3]:
# Set up the remote dask cluster. Can either use this version or a similar version above if building a LocalCluster.
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,memory="32GB")
client = Client(cluster)
cluster.scale(cores=2)
#cluster.adapt(minimum=2, maximum=16)
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.131:44327,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
#cluster.scale(cores=2)

In [5]:
bgc=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_bgc_month.zarr.zip')
bgc=bgc.rename({'xt_ocean':'lon','yt_ocean':'lat'})
bgc['lon']=bgc['lon']+360
eqpac=bgc.sel(lon=slice(120,290),lat=slice(-20,20))
#eqpac.time
eqpac
eqpac['time']=np.array(eqpac.indexes['time'].to_datetimeindex(), dtype='datetime64[M]')

/local/v45/np1383/tmp/ipykernel_2885753/2605875320.py:7: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  eqpac['time']=np.array(eqpac.indexes['time'].to_datetimeindex(), dtype='datetime64[M]')


In [6]:
phys=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_month.zarr.zip')
phys=phys.rename({'xu_ocean':'lon_x','yu_ocean':'lat_x'})
phys=phys.rename({'xt_ocean':'lon','yt_ocean':'lat'})

phys['lon']=phys['lon']+360
phys['lon_x']=phys['lon_x']+360
phys['st_ocean']=phys['st_ocean']*-1
phys['sw_ocean']=phys['sw_ocean']*-1
phys_eqpac=phys.sel(lon=slice(120,290),lat=slice(-20,20))

In [7]:
#eqpac_use=eqpac.stf10.chunk(dict(time=-1)).sel(ensemble=26)
#eqpac_use=eqpac_use.interpolate_na(dim='time')#,dask_gufunc_kwargs={'allow_rechunk':True})
#eqpac_use=xr.open_dataset('../processed_data/eof_eqpac_test.nc')

In [8]:
#eqpac_use

In [9]:
#solver=Eof(eqpac_use.stf10)

In [10]:
#eqpac_use.to_netcdf('../processed_data/eof_eqpac_test.nc')

In [7]:
key_drivers=['paco2','stf10','pprod_gross_2d','surface_alk']#pco2#
key_drivers_phys=['sst','salt']

vars_of_interest=eqpac[key_drivers]
vars_of_interest[key_drivers_phys]=phys_eqpac[key_drivers_phys]

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...   

In [8]:
vars_of_interest=vars_of_interest.sel(ensemble=26,st_ocean=-5)

In [9]:
vars_of_interest

<xarray.Dataset>
Dimensions:         (time: 720, lat: 98, lon: 170)
Coordinates:
  * time            (time) datetime64[ns] 1960-01-01 1960-02-01 ... 2019-12-01
    ensemble        int32 26
  * lon             (lon) float64 120.5 121.5 122.5 123.5 ... 287.5 288.5 289.5
  * lat             (lat) float64 -19.36 -18.59 -17.85 ... 17.85 18.59 19.36
    st_ocean        float64 -5.0
Data variables:
    paco2           (time, lat, lon) float32 dask.array<chunksize=(1, 98, 170), meta=np.ndarray>
    stf10           (time, lat, lon) float32 dask.array<chunksize=(1, 98, 170), meta=np.ndarray>
    pprod_gross_2d  (time, lat, lon) float32 dask.array<chunksize=(1, 98, 170), meta=np.ndarray>
    surface_alk     (time, lat, lon) float32 dask.array<chunksize=(1, 98, 170), meta=np.ndarray>
    sst             (time, lat, lon) float32 dask.array<chunksize=(720, 98, 170), meta=np.ndarray>
    salt            (time, lat, lon) float32 dask.array<chunksize=(720, 98, 170), meta=np.ndarray>
Attributes: (12/24)
    calendar:                 julian
    cm-enkf_source:          commit 0141830a243704acbf6d8c8c843b161a39ec39fa
    contact_name:            Decadal Activity 1 - Data Assimilation
    control_name:            c5
    creation_date:           Sun Dec  1 23:38:56 2019
    data_assimilation_name:  d60
    ...                      ...
    model_source:            commit 2abb29f4384e68777721fa657850587f28efd85a
    nominal_resoltuion:      Atmosphere delta lat = 2.02degrees ; Atmosphere ...
    perturbation_name:       not applicable
    references:              OKane, T.J., Sandery, P.A., Monselesan, D.P., Sa...
    run_variant_name:        data assimilation
    title:                   AccessOcean-AM2

In [14]:
#vars_of_interest.variables

In [15]:
cluster

In [16]:
single_pixel=False
if single_pixel==True:
    lalos=[[0,180],
           [0,270],
           [-10,270],
           [10,270],
           [0,220]]

    # Try a single pixel
    for la,lo in lalos:

        pixel=vars_of_interest.sel(lat=la,lon=lo,method='nearest')

        pix_df_raw=pixel.to_dataframe()
         # This one takes a second or 1000..
        pix_df=pix_df_raw[1:].drop(['lat','lon','ensemble','st_ocean'],axis=1)
        cols=list(pix_df)
        y=cols[0]
        x=cols[1:]
        kru=Kruskals.from_pandas_df(pix_df, x, y).driver_score_to_series(directional=True)
        print(kru)

In [17]:
# THIS WORKS for a single pixel
single+pixe
if single_pixel==True:
    cutout=data.sel(x=2,y=2)
    cutout_pd=cutout.to_pandas().drop(['x','y'],axis=1)
    #labels
    cols=list(cutout_pd)
    y=cols[0] # To tell Kruskals which columns to use. Can just just Kruskals() direct with np arrays instead though.
    x=cols[1:] 

    kru=Kruskals.from_pandas_df(cutout_pd, x, y).driver_score_to_series()
    print(kru)

In [14]:
import pingouin as pg

In [ ]:
pixel=vars_of_interest.sel(lat=0,lon=180,method='nearest')

pix_df_raw=pixel.to_dataframe()
 # This one takes a second or 1000..
pix_df=pix_df_raw[1:].drop(['lat','lon','ensemble','st_ocean'],axis=1)
cols=list(pix_df)
y=cols[0]
x=cols[1:]

In [ ]:
pg.linear_regression(pix_df[x], pix_df[y],relimp=True)

In [ ]:
def test_apply_relimp(a,b,c,d,e):
    #print(np.array([b,c,d,e]))

    drivers=Kruskals(np.array([b,c,d,e]),np.array(a))
    
    m_linear_regression=pg.linear_regression(pix_df[x], pix_df[y],relimp=True)
    return m_linear_regression

In [10]:
small_reg=vars_of_interest.sel(lat=slice(-2,0),lon=slice(180,182))
small_reg=small_reg.chunk(dict(time=-1))

In [11]:
vars_of_interest=vars_of_interest.chunk(dict(time=-1))

In [18]:
# I think this solution is close but it sti

def test_apply_kruskals(a,b,c,d,e):
    #print(np.array([b,c,d,e]))

    drivers=Kruskals(np.array([b,c,d,e]),np.array(a))
    
    driver_score=drivers.driver_score(directional=True)
    return driver_score

In [18]:
dr=xr.apply_ufunc(test_apply_relimp,
                  small_reg['paco2'], # Independent Variable
                  small_reg['pprod_gross_2d'], # Depends Vars
                  small_reg['surface_alk'], # Depends Vars
                  small_reg['sst'], # Depends Vars
                  small_reg['salt'], # Depends Vars
                  input_core_dims=[["time"],["time"],["time"],["time"],["time"]],
                  dask='parallelized',
                  vectorize=False,
                  output_core_dims=[['drivers']],
                  output_dtypes=['float64'],
                  dask_gufunc_kwargs={'output_sizes':{"drivers": 4}})

In [19]:
dr.to_netcdf('../processed_data/kruskals_test_drivers_smallreg.nc')

KilledWorker: ("('getitem-b7e9b074bf26d72164259c1be2eb4d27', 0, 0, 0, 0, 0)", <WorkerState 'tcp://10.0.128.11:44049', name: SLURMCluster-5-1, memory: 0, processing: 5>)

In [20]:
dr

<xarray.DataArray (lat: 6, lon: 2, drivers: 4)>
dask.array<transpose, shape=(6, 2, 4), dtype=float64, chunksize=(6, 2, 4), chunktype=numpy.ndarray>
Coordinates:
    ensemble  int32 26
  * lon       (lon) float64 180.5 181.5
  * lat       (lat) float64 -1.834 -1.5 -1.167 -0.8329 -0.5005 -0.1662
    st_ocean  float64 -5.0
Dimensions without coordinates: drivers

In [ ]:
small_reg.paco2.mean(dim=['lat','lon']).plot()
small_reg.pprod_gross_2d.mean(dim=['lat','lon']).plot()
small_reg.surface_alk.mean(dim=['lat','lon']).plot()
small_reg.sst.mean(dim=['lat','lon']).plot()
small_reg.salt.mean(dim=['lat','lon']).plot()

In [ ]:
dr.sel(drivers=1).plot()

In [ ]:
# SMall region test

In [ ]:
data

In [ ]:
dr=xr.apply_ufunc(test_apply_kruskals,
                  small_reg['paco2'],
                  small_reg['stf10'],
                  small_reg['sst'],
                  small_reg['salt'],
                  small_reg['pprod_gross_2d'], input_core_dims=[["time"],["time"],["time"],["time"],["time"]],dask='allowed',vectorize=True,output_core_dims=[['drivers']])

In [ ]:
kru=Kruskals.from_pandas_df(pix_df, x, y).driver_score_to_series()

In [ ]:
kru

In [ ]:
# SMall region test
small_reg=vars_of_interest.sel(lat=slice(-2,0),lon=slice(180,182))

In [ ]:
#xr.apply_ufunc(test_apply_kruskals,small_reg.paco2,small_reg.sst,small_reg.stf10,dask='allowed',vectorize=True)#,input_core_dims=[['lat'],['lat'],['lat']])
#small_reg.paco2.to_dataframe()
small_reg

In [ ]:
def test_apply_kruskals(a,b,c,d,e):
    #print(a.sum().values)
    print(a.shape)
    print(np.asarray(a).shape)
    #print(df[1])
    drivers=Kruskals(a,(b,c,d,e))
    print('RUN')
    drivers_series=drivers.driver_score_to_series()
    return drivers_series#[0.5,0.2,0.2,0.1,0.1]
    
    #return drivers
#    return drivers

#dask.array.apply_along_axis(test_apply_kruskals,'time',small_reg)

In [ ]:
#small_reg=small_reg.chunk({'time':len(small_reg.time),'lat':1,'lon':1})
dr=xr.apply_ufunc(test_apply_kruskals,
                  small_reg['paco2'],
                  small_reg['stf10'],
                  small_reg['sst'],
                  small_reg['salt'],
                  small_reg['pprod_gross_2d'], input_core_dims=[["time"],["time"],["time"],["time"],["time"]],dask='allowed',vectorize=True,output_core_dims=[['drivers']])#.paco2.values

In [ ]:
%debug

In [ ]:
#applyask.array.apply_ufunc(test_apply_kruskals,' ()->() ',small_reg, output_dtypes=(float))
dr=xr.apply_ufunc(test_apply_kruskals,
                  small_reg['paco2'],
                  small_reg['stf10'],
                  small_reg['sst'],
                  small_reg['salt'],
                  small_reg['pprod_gross_2d'],
                  dask='allowed',vectorize=True,input_core_dims=[['lat'],['lat'],['lat'],['lat'],['lat']])#,input_core_dims=[['lat','lon'],['lat','lon'],['lat','lon'],['lat','lon'],['lat','lon']])#
                                                                                                                                #exclude_dims=set(('time',)))


#(test_apply_kruskals),axis='time')

In [ ]:
pix_df.T.to_numpy()[0]

In [ ]:
vars_of_interest

In [ ]:
pix_df.to_numeric()

In [ ]:
corr_matrix=pix_df.apply(pd.to_numeric,errors='coerce').corr()
corr_X = corr_matrix.iloc[1:, 1:].copy()
corr_Xy = corr_matrix.iloc[1:, 0].copy()

In [ ]:
corr_X

In [ ]:
np.linalg.eig(pix_df)

In [ ]:
w_corr_Xs, v_corr_Xs = np.linalg.eig(corr_Xs)
diag_idx = np.diag_indices(len(corr_Xs))
diag = np.zeros((num_drivers, num_drivers), float)
diag[diag_idx] = w_corr_Xs
delta = np.sqrt(diag)

In [ ]:
coef_xz = v_corr_Xs @ delta @ v_corr_Xs.transpose()
coef_yz = np.linalg.inv(coef_xz) @ corr_Xy

In [ ]:
r2 = sum(np.square(coef_yz))
raw_relative_weights = np.square(coef_xz) @ np.square(coef_yz)
normalized_relative_weights = (raw_relative_weights/r2)*100

In [ ]:
cluster()

In [ ]:
print('hello test')

In [ ]:
# Load example Imports
from Kruskals import Kruskals #https://github.com/Rambatino/Kruskals
import numpy as np
import xarray as xr

# Make Fake Data
times = pd.date_range('2019-01-01', '2019-01-10', name='time')
y=np.arange(0,8)
x=np.arange(0,8)
data1 = xr.DataArray(np.random.rand(10, 8, 8), dims=["time", "y", "x"], coords={'time': times,'x':x,'y':y},name='data1')
data2 = xr.DataArray(np.random.rand(10, 8, 8), dims=["time", "y", "x"], coords={'time': times,'x':x,'y':y},name='data2')
data3 = xr.DataArray(np.random.rand(10, 8, 8), dims=["time", "y", "x"], coords={'time': times,'x':x,'y':y},name='data3')
data4 = xr.DataArray(np.random.rand(10, 8, 8), dims=["time", "y", "x"], coords={'time': times,'x':x,'y':y},name='data4')
data5 = xr.DataArray(np.random.rand(10, 8, 8), dims=["time", "y", "x"], coords={'time': times,'x':x,'y':y},name='data5')

data=xr.merge([data1,data2,data3,data4,data5])
data = data.chunk({'time':10, 'x':1, 'y':1})

In [ ]:
# This method works for a single pixel
cutout=data.sel(x=2,y=2)
cutout_pd=cutout.to_pandas().drop(['x','y'],axis=1)
#labels
cols=list(cutout_pd)
y=cols[0] # To tell Kruskals which columns to use. Can just just Kruskals() direct with np arrays instead though.
x=cols[1:] 

kru=Kruskals.from_pandas_df(cutout_pd, x, y).driver_score_to_series()
print(kru)

# But have tried a few hours and can't work out how to vectorise this

In [ ]:
cutout_pd

In [ ]:
data.data1.to_numpy()

In [ ]:
dependent=np.array([data2.to_numpy()[:,1,1],
          data3.to_numpy()[:,1,1],
          data4.to_numpy()[:,1,1],
          data5.to_numpy()[:,1,1]])
independent=data1.to_numpy()[:,1,1]

In [ ]:
#cutout_np=cutout_pd.to_numpy()
k=Kruskals(dependent,independent)
k.driver_score()

In [ ]:
dependent

In [ ]:
cutout_np[0].shape

In [ ]:
def test_apply_kruskals(a,b,c,d,e):
    #print(np.array([b,c,d,e]))
    print(a.shape)
    drivers=Kruskals(np.array([b,c,d,e]),np.array(a))
    
    driver_score=drivers.driver_score()
    return driver_score

dr=xr.apply_ufunc(test_apply_kruskals,
                  data['data1'], # Independent Variable
                  data['data2'], # Depends Vars
                  data['data3'], # Depends Vars
                  data['data4'], # Depends Vars
                  data['data5'], # Depends Vars
                  input_core_dims=[["time"],["time"],["time"],["time"],["time"]],
                  dask='parallelized',
                  vectorize=True,
                  output_core_dims=[['drivers']],
                  output_dtypes=['float64'],
                  output_sizes={"drivers": 4})
# Throws TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
dr

In [ ]:
%debug

In [ ]:
np.array(data['data1'],data['data2'])

In [ ]:
data['data1']

In [ ]:
#test linregress
from scipy.stats import linregress

def new_linregress(x, y):
    # Wrapper around scipy linregress to use in apply_ufunc
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
   
    print(x)
    print(y)
    return np.array([slope, intercept, r_value, p_value, std_err])
# return a new DataArray
stats = xr.apply_ufunc(new_linregress, data['time'], data['data2'],
                       input_core_dims=[['time'], ['time']],
                       output_core_dims=[["parameter"]],
                       vectorize=True,
                       dask="parallelized",
                       output_dtypes=['float64'],
                       dask_gufunc_kwargs={'output_sizes':{"parameter": 5}})
#data['parameter']=stats
data

In [ ]:
stats

In [ ]:
stats

In [ ]:
cluster

In [ ]:
#test linregress
from scipy.stats import linregress

def new_linregress(x, y):
    # Wrapper around scipy linregress to use in apply_ufunc
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return np.array([slope, intercept, r_value, p_value, std_err])

# return a new DataArraya
stats = xr.apply_ufunc(new_linregress, data['time'], data['data2'],
                       input_core_dims=[['time'], ['time']],
                       output_core_dims=[["parameter"]],
                       vectorize=True,
                       dask="parallelized",
                       output_dtypes=['float64'],
                       output_sizes={"parameter": 5})
data['parameter']=stats
data